<a href="https://www.kaggle.com/code/nikoladyulgerov/rainfall?scriptVersionId=225786296" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
SEED = 45

## Descripción general de los datos

Leemos los archivos en formato `.csv` proporcionados por la competición. Trabajaremos principalmente con el conjunto de entrenamiento, pero siempre es bueno echar un vistazo a cómo es el conjunto de test por si hay alguna discrepancia.

**NOTA** El directorio puede variar según el entorno donde se ejecute el notebook

In [3]:
df_train = pd.read_csv("/kaggle/input/playground-series-s5e3/train.csv", index_col="id")
df_test = pd.read_csv("/kaggle/input/playground-series-s5e3/test.csv", index_col="id")

In [4]:
df_train.sample(5)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
id,,,,,,,,,,,,
712,348,1023.9,17.6,15.2,11.5,13.6,93.0,88.0,0.2,40.0,17.3,1
1497,38,1020.2,21.2,18.9,17.2,15.0,74.0,35.0,9.8,30.0,18.6,0
1942,118,1010.3,26.5,25.9,24.3,22.4,84.0,80.0,3.5,80.0,21.6,1
854,125,1007.6,30.4,26.3,23.2,23.3,89.0,66.0,7.7,75.0,8.5,1
1613,154,1008.5,34.2,30.1,28.7,24.4,72.0,22.0,10.5,240.0,9.5,0


In [5]:
df_test.sample(5)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
id,,,,,,,,,,,
2252,63,1019.3,23.1,21.8,18.2,18.1,95.0,91.0,0.0,40.0,16.0
2833,279,1017.9,30.5,27.7,25.6,23.2,84.0,61.0,4.6,210.0,11.9
2313,124,1012.0,30.8,26.4,24.0,19.9,76.0,61.0,8.6,10.0,11.8
2333,144,1005.7,29.8,26.5,25.1,24.4,85.0,81.0,0.1,60.0,7.9
2286,97,1009.5,28.6,26.8,25.8,23.3,89.0,88.0,0.3,220.0,30.3


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2190 entries, 0 to 2189
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            2190 non-null   int64  
 1   pressure       2190 non-null   float64
 2   maxtemp        2190 non-null   float64
 3   temparature    2190 non-null   float64
 4   mintemp        2190 non-null   float64
 5   dewpoint       2190 non-null   float64
 6   humidity       2190 non-null   float64
 7   cloud          2190 non-null   float64
 8   sunshine       2190 non-null   float64
 9   winddirection  2190 non-null   float64
 10  windspeed      2190 non-null   float64
 11  rainfall       2190 non-null   int64  
dtypes: float64(10), int64(2)
memory usage: 222.4 KB


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 730 entries, 2190 to 2919
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            730 non-null    int64  
 1   pressure       730 non-null    float64
 2   maxtemp        730 non-null    float64
 3   temparature    730 non-null    float64
 4   mintemp        730 non-null    float64
 5   dewpoint       730 non-null    float64
 6   humidity       730 non-null    float64
 7   cloud          730 non-null    float64
 8   sunshine       730 non-null    float64
 9   winddirection  729 non-null    float64
 10  windspeed      730 non-null    float64
dtypes: float64(10), int64(1)
memory usage: 68.4 KB


Parece que el **conjunto de test** tiene un valor perdido para la variable `winddirection`, habrá que tratarlo de alguna forma antes de realizar cualquier predicción. Además la variable objetivo `rainfall` no está presente ya que es la que debemos predecir con nuestros modelos

In [8]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
day,2190.0,179.948402,105.203592,1.0,89.000,178.50,270.000,365.0
pressure,2190.0,1013.602146,5.655366,999.0,1008.600,1013.00,1017.775,1034.6
maxtemp,2190.0,26.365799,5.654330,10.4,21.300,27.80,31.200,36.0
temparature,2190.0,23.953059,5.222410,7.4,19.300,25.50,28.400,31.5
mintemp,2190.0,22.170091,5.059120,4.0,17.700,23.85,26.400,29.8
dewpoint,2190.0,20.454566,5.288406,-0.3,16.800,22.15,25.000,26.7
humidity,2190.0,82.036530,7.800654,39.0,77.000,82.00,88.000,98.0
cloud,2190.0,75.721918,18.026498,2.0,69.000,83.00,88.000,100.0
sunshine,2190.0,3.744429,3.626327,0.0,0.400,2.40,6.800,12.1
winddirection,2190.0,104.863151,80.002416,10.0,40.000,70.00,200.000,300.0


* **Day** -> de 1 hasta 365 que son los días que tiene un año. Al haber 2190 registros, estimamos que los datos se registraton durante 6 años
* **Pressure** -> por los valores se puede deducir que las medidas se han tomado en *hPa* (hectopascales)
* **MaxTemp, MinTemp y Temperature** -> valores máximos, mínimos y medios de temperatura tomados en *°C* (grados celsius)
* **DewPoint** -> punto de rocío (temperatura a la que el agua se condensa con el enfriamiento) medida en *°C* (grados celsius). Técnicamente se puede calcular con la temperatura y la humedad relativa
* **Humidity** -> por los valores se puede suponer que se trata de los porcentajes de humedad relativa dado que no supera el 100%. Ligada a la temperatura y el punto de rocío
* **Cloud** -> nubosidad es la fracción de cielo cubierto con nubes, de ahi que sea expresada también en porcentajes (valor máximo 100%)
* **Sunshine** -> horas de sol hasta un máximo de 12. *Resulta extraño que el mínimo sea 0 dado que es un fenómeno llamado noche polar*
* **WindDirection** -> dirección del viento que entendemos parcialmente si es expresada en grados. los valores son muy exactos, quizás habrá que tratarla como variable categórica y no como numérica
* **WindSpeed** -> velocidad del viento en *Km/H* (kilometros/hora) dado los valores entre los que oscila
* **RainFall** -> Llueve o no llueve expresado con *one-hot encoding*. Entendemos como `1 == llueve` y `0 == no llueve`

Aspectos a tener en cuenta:
- A priori, las variables `cloud` y `windspeed` son las que más desviación estándar presentan, lo cual las hace buenas candidatas para ser relevantes a la hora de clasificar
- La variable objetivo `rainfall` no está balanceada (`media == 0.75`), habrá que escoger los algoritmos y las métricas adecuadas para este tipo de problemas
- En un principio, un paso fundamental en el procesamiento de los datos será **escalarlos**

In [9]:
df_train['winddirection'].unique()

array([ 60. ,  50. ,  70. ,  40. ,  20. ,  30. ,  80. ,  90. , 220. ,
       100. , 290. , 170. , 200. , 230. , 240. , 130. , 270. , 120. ,
       190. , 210. , 110. , 160. ,  10. , 180. , 280. , 250. , 300. ,
       260. ,  25. ,  75. , 150. , 140. ,  15. , 250.3,  65. ])

In [10]:
print(f"Duplicados en conjunto de entrenamiento: {df_train.duplicated().sum()}")
print(f"Duplicados en conjunto de test: {df_test.duplicated().sum()}")

Duplicados en conjunto de entrenamiento: 0
Duplicados en conjunto de test: 0


- Gráficas por cada variable -> outliers, desviación, skew

## Análisis Exploratorio

En esta sección visualizaremos y analizaremos los datos a fondo para encontrar características que nos interesen con el fin de escoger modelos que se ajusten mejor.

In [11]:
num_cols = df_train.select_dtypes(include='float').columns.to_list()
# cat_cols = []
target = "rainfall"
num_cols

['pressure',
 'maxtemp',
 'temparature',
 'mintemp',
 'dewpoint',
 'humidity',
 'cloud',
 'sunshine',
 'winddirection',
 'windspeed']

In [12]:
fig = px.histogram(df_train, x=target, color=target, barmode='group')
fig.show()

Nos encontramos que efectivamente el desbalance de la clase es sustancial dado que la mayoría de registros que se tienen indican que hubo precipitaciones. 

**NOTA** En caso de realizar particiones de los datos deberán ser estratificadas. Métricas como la curva ROC o F1-score se ajustan mejor a este tipo de problemas

In [13]:
rainfall_classes = df_train["rainfall"].unique()
color_map = px.colors.qualitative.Plotly[:len(rainfall_classes)]
rainfall_color_dict = dict(zip(rainfall_classes, color_map))  # Map colors to categories

for col in num_cols:
    
    # Create a figure with 1 row, 2 columns
    fig = make_subplots(rows=1, cols=2, subplot_titles=[f"Boxplot of {col}", f"Histogram of {col}"])

    # Boxplot (Grouped by Rainfall)
    for category in rainfall_classes:
        fig.add_trace(
            go.Box(
                y=df_train[df_train["rainfall"] == category][col],
                name=f"Rainfall: {category}",
                marker_color=rainfall_color_dict[category],
                boxpoints="all",
                jitter=0.5,
                pointpos=-2
            ),
            row=1, col=1
        )

    # Overlayed Histogram (Grouped by Rainfall)
    for category in rainfall_classes:
        fig.add_trace(
            go.Histogram(
                x=df_train[df_train["rainfall"] == category][col],
                name=f"Rainfall: {category}",
                marker_color=rainfall_color_dict[category],  # Apply correct color
                opacity=0.6,
                showlegend=False
            ),
            row=1, col=2
        )

    # Update layout
    fig.update_layout(
        showlegend=True, barmode="overlay"  # Overlapping histograms
    )

    fig.show()

- La presión sigue una distribución normal
- Las variables de temperatura y punto de rocío tienen una asimetría negativa (hacia la izquierda) 
- Es claro el potencial para clasificar de las variables `sunshine`, `cloud` y `humedity`. Tanto en los histogramas como en los diagramas de cajas se observa que a partir de cierto valor llueve siempre o todo lo contrario.
- La velocidad del viento `windspeed` tiene una asimetría positiva, pero al igual que las temperaturas no hay distincion en la distribución de la variable clase 
- La variable `winddirection` presenta una distribución bimodal congregandose alrededor de valores fijos (~50° y ~220°)

**NOTA** Algunas variables contienen unos cuantos `outliers`, los cuales deberan ser tratados debidamente a la hora de escalar los datos en el procesamiento

In [14]:
fig = px.imshow(df_train[num_cols].corr().round(2), text_auto=True)
fig.show()

- Las variables de temperatura están correlacionadas entre sí, algo obvio y redundante. Se podría prescindir de alguna de ellas.
- Vemos que las variables de temperatura (`max`, `min`, `temparature`) y punto de rocío (`dewpoint`) están altamente correlacionadas, dado que como hemos mencionado para conocer el punto de rocío es necesario saber la temperatura
- La humedad y la nubosidad tambien presentan cierta relacion entre si. A más nubes, más humedad, algo lógico
- La presión y la temperatura al igual que nubosidad (`cloud`) y horas de luz (`sunshine`) presentan una correlación negativa, lo que indica que actúan como variables opuestas: cuando una crece, la otra disminuye.

## Procesamiento de los datos 

Procedemos a preparar los datos para que puedan ser ingestados por los modelos que vamos a construir. 
Atenderemos a todas las conclusiones sacadas de la exploración y aplicaremos las técnicas pertinentes

Empezamos por imputar el **único** valor perdido en el conjunto de test con la **mediana** al funcionar mejor con datos asimétricos que por ejemplo la media.

**RECUERDA** Esto no es una fuga de datos dado que tenemos los conjuntos separados. La operación la realizamos de manera ad-hoc porque no vale la pena poner mayor esfuerzo para un solo valor. En caso de tener un mayor porcentaje de valores perdidos, habría que optar por métodos mas robustos

In [15]:
df_test['winddirection'] = df_test['winddirection'].fillna(df_test['winddirection'].median())
df_test['winddirection'].isna().sum()

0

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [17]:
def feature_engineering(df):
    
    # Nuevas variables numéricas
    df['temp_range'] = df['maxtemp'] - df['mintemp']

    # Nuevas variables categóricas
    df['month'] = pd.to_datetime(df['day'], format='%j').dt.month 
    df['season'] = df['month'].apply(lambda x: 1 if 3 <= x <= 5  # Primavera
                                         else 2 if 6 <= x <= 8  # Verano
                                         else 3 if 9 <= x <= 11  # Otoño
                                         else 0)  # Invierno
    df['wind_cardinal'] = pd.cut(df['winddirection'], bins=[0, 90, 180, 270, 360], labels=['NE', 'SE', 'SW', 'NW'], include_lowest=True)

    # Borrar variabels redundantes
    df = df.drop(columns=['maxtemp', 'mintemp', 'winddirection'])
    
    return df

In [18]:
df_train = feature_engineering(df_train)
df_test = feature_engineering(df_test)

In [19]:
df_train.sample(5, random_state=SEED)

,day,pressure,temparature,dewpoint,humidity,cloud,sunshine,windspeed,rainfall,temp_range,month,season,wind_cardinal
id,,,,,,,,,,,,,
1914,90,1008.5,25.4,24.2,94.0,85.0,1.5,14.2,1,3.6,3,1,NE
1254,160,1009.5,30.1,26.1,75.0,61.0,7.2,9.2,1,4.7,6,2,SE
2148,324,1017.0,20.4,18.6,95.0,83.0,0.0,24.8,1,2.6,11,3,NE
1128,34,1019.5,20.9,16.6,79.0,47.0,6.8,9.1,1,3.8,2,0,NE
1196,102,1012.5,25.2,22.9,93.0,88.0,0.1,13.4,0,1.6,4,1,NE


In [20]:
cat_cols = ['wind_cardinal', 'month', 'season']
num_cols = df_train.select_dtypes(include='float').columns.to_list()

In [21]:
X = df_train.drop(['rainfall'], axis=1)
y = df_train['rainfall']

Al tener conjuntos de entrenamiento y test por separado, no es necesario realizar un **holdout**

In [22]:
num_transformer = Pipeline([('rb_scaler', RobustScaler())])	
num_transformer

Pipeline(steps=[('rb_scaler', RobustScaler())])

In [23]:
cat_transformer = Pipeline([
        ('one_hot', OneHotEncoder(handle_unknown='ignore'))
])
cat_transformer

Pipeline(steps=[('one_hot', OneHotEncoder(handle_unknown='ignore'))])

In [24]:
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
])
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('rb_scaler',
                                                  RobustScaler())]),
                                 ['pressure', 'temparature', 'dewpoint',
                                  'humidity', 'cloud', 'sunshine', 'windspeed',
                                  'temp_range']),
                                ('cat',
                                 Pipeline(steps=[('one_hot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['wind_cardinal', 'month', 'season'])])

Ya tenemos automatizado el pipeline de preprocesamiento, cualquier transformación que sea necesaria añadir a posteriori se hará de forma más cómoda

## Evaluación de modelos

In [25]:
dtree_model = DecisionTreeClassifier(max_depth=5, random_state=SEED)

pipe_dtree = Pipeline([
    ('prep', preprocessor),
    ('clas', dtree_model)
])

In [26]:
parameters = {}
parameters['clas__max_depth'] = [None, 3, 5, 7, 9]
# parameters['clas__class_weight'] = [None, 'balanced']

GS = GridSearchCV(pipe_dtree, parameters, cv=5, scoring='roc_auc', refit=True)

GS.fit(X, y)

best_pipe_tree = GS.best_estimator_

print("Mejor score: ", GS.best_score_)
print("Mejore configuración de parámetros: ", GS.best_params_)

Mejor score:  0.8569809203142537
Mejore configuración de parámetros:  {'clas__max_depth': 3}


In [27]:
from sklearn.metrics import roc_curve, auc

In [28]:
y_score = best_pipe_tree.predict_proba(X)[:, 1]
fpr, tpr, thresholds = roc_curve(y, y_score)

In [29]:
fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

In [30]:
test_preds = best_pipe_tree.predict_proba(df_test)[:, 1]

submission = pd.DataFrame({'id': df_test.index, 'rainfall': test_preds})
submission.to_csv("submission.csv", index=False)
print("\nFichero a enviar guardado como 'submission.csv'.")


Fichero a enviar guardado como 'submission.csv'.
